In [ ]:
#| default_exp wind_res


In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd

In [ ]:
#| export
import numpy as np
import pandas as pd
import pkg_resources
from fastcore.test import *

# Wind resistance factors

The 

In [ ]:
'''
#| export
def load_wind_factors(vessel_type:str #The name of the vessel type. Must be one of 9 options
                     ):
    
    "Loads a data frame of the wind factors for a range of angles of attack for the named vessel type"
    
    stream = pkg_resources.resource_stream('data/', vessel_type+'.csv')
    return pd.read_csv(stream, encoding='latin-1')
    
''' 
    

'\n#| export\ndef load_wind_factors(vessel_type:str #The name of the vessel type. Must be one of 9 options\n                     ):\n    \n    "Loads a data frame of the wind factors for a range of angles of attack for the named vessel type"\n    \n    stream = pkg_resources.resource_stream(\'data/\', vessel_type+\'.csv\')\n    return pd.read_csv(stream, encoding=\'latin-1\')\n    \n'

In [ ]:
#load_wind_factors('280_KDWT_TANKER')

ModuleNotFoundError: No module named 'data/'